# Data Normalization Mini Class

In [ ]:
import os

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

from data_norm_class.constants import (
    FILE_DATA_RAW,
    FOLDER_DATA_NORMALIZED,
    FOLDER_DB
)
from data_norm_class.io import (
    read_raw_aarc_data
)
from data_norm_class.normalization import (
    normalize_data
)

In [ ]:
FOLDER_DATA_NORMALIZED = os.path.join("..", FOLDER_DATA_NORMALIZED)
FILE_DATA_RAW = os.path.join("..", FILE_DATA_RAW)
FOLDER_DB = os.path.join("..", FOLDER_DB)

## What's wrong with single CSV files?

- CSV files are a great way to store your tabular data without much memory overhead
- The tabular structure is flexible and CSVs are accessible across domains and tools
- It's a great way to store data in which each row is a unique **entity**
- If this uniqueness is violated, things easily become messy

Let's use a random subset of 40 departments of the `AARC-v3` dataset as our sample.

In [ ]:
aarc_sample_raw = read_raw_aarc_data(FILE_DATA_RAW)
aarc_sample_raw.head()

On a first glance, the file structure might suggest that each row stands for a faculty (identified by `PersonId`) being hired at a department (`DepartmentId`) in a given year (`Year`) with additional metadata.

Let's check this assumption by comparing the number of unique rows for these triplets.

In [ ]:
print((
    "Number of unique PersonId, Year, and DepartmentId combinations: "
    f"{aarc_sample_raw[['PersonId', 'Year', 'DepartmentId']].drop_duplicates().shape[0]}\n"
    f"Number of rows in the raw data: {aarc_sample_raw.shape[0]}"))


The AARC dataset can be considered as **relational data**, which consists of
- **entities** refer to individually identifiable objects (e.g., faculty members, departments, fields, ...)
- **relationships** are connections between entities (e.g., faculty being hired at a department, a department being assigned to a field in a given year)
- **attributes** of entities or relationships that provide contextual information (e.g., the year of faculty hiring or a faculty member's name) 

There is a mismatch of roughly 4,000 rows, because the dataset contains the all combinations of multiple **entities**, all of their **relationships** and **attributes**.
This leads to multiple problems.

### Redundancy
- Much of the information is **redundant** across rows
  - for instance, consider the `DepartmentName` column

In [ ]:
aarc_sample_raw["DepartmentName"].nunique()

This column stores only 34 different values because it only holds a single information per department.
However, these values are repeated across all 12k rows, which is very memory-inefficient.

In [ ]:
print(("Memory usage of the 'DepartmentName' column: "
       f"{aarc_sample_raw['DepartmentName'].memory_usage(index=False, deep=True) / 1024**2:.2f} MB"))

Recall that this is just a small sample of the full data. 
The entire original dataset contains ~916MB of data which can quickly increase when performing join-operations.
Normalizing the data will improve this by assigning `DepartmentName`s only to the entity that it's relevant for (departments) and isolating this relationship from others in separate tables.

Redundancy also entails repeated relationships.
As we've seen before, each row holds a combination of all possible relationships between the columns `(DepartmentId, PersonId, Year, Taxonomy)`.
This way, all `(DepartmentId, PersonId, Year)` hirings are repeated multiple times if a department is assigned to multiple umbrellas. 
This is not only inefficient, but prone to errors when aggregating data based on these attributes.

Another problem with redundant data are the costs of alteration.
Updates always affect all entries in our entire dataset.
If updates are common, e.g., in databases storing users, this can quickly become a bottleneck, slowing down other operations.
Adding other relationships, e.g., publication data, requires joining the entire data.
In this example, each `(PersonId, Year)` item would be linked to all its publications.
Because `(PersonId, Year)` is already redundant, this would greatly increase the dataset size.


### Integrity

- each row in AARC is **uniquely identfiable** only by a tuple of `(DepartmentId, PersonId, Year, Taxonomy)`
  - many entities (like domains) and relationships (faculty hiring) are not directly identifiable
- **entity integrity** ensures that entities are identifiable
  - they must have a **primary key**: a unique identifier differentiating them from each other
  - no two entities should have the same primary key and it should not be missing
  - the `AARCv3` data already has these identifiers for faculty (`PersonId`), departments (`DepartmentId`) and institutions (`InstitutionId`)
    - this makes it possible to differentiate between entities with similar attributes, e.g., people of the same name

In [ ]:
aarc_sample_raw[["PersonId", "DepartmentId", "InstitutionId"]]\
    .nunique()

- however, there are no explicit identifiers for relationships
- **foreign integrity** ensures that relationships link to valid entities
  - for this purpose, a **foreign key** maps between items in one table to existing entities in another table
    - for instance, if we had a table of publication records with `PersonId` as a foreign key, these should point to a single, valid faculty member
- making relationships identifiable allows us to easily extend the data by assigning metadata
  - for instance, assigning faculty `Rank` to their hiring at a given year  
- we will need to ensure this integrity when splitting our dataset into  multiple tables in the normalization process

## Data normalization
- data normalization ensures non-redundancy and integrity in entities and relations
- by isolating entities from their relations, it improves 
  - expressiveness
  - size requirements
  - extendibility
- various **normal forms** pose increasing demands
- typically, a dataset is consider normalized, if it follows the third normal form **3NF**

### 3. Normal Form (3NF)

3NF requires that 
1. each cell only stores an atomic value (no lists or other collections like `json`)
2. all non-key columns (or **attributes**) depend on the entire primary key, not just parts of it
3. there are no transitive relationships in which one attribute depends on another 

Let's have a look at the columns of `AARC-v3`.

In [ ]:
aarc_sample_raw.columns

#### 3NF violations of AARC-v3

- `AARC-v3` does not violate the first requirement (all columns contain atomic values only)
- the **primary key** of this table is a **compound key** of 
  - `PersonId`
  - `DepartmentId`
  - `Year`
  - `Umbrella`
  

In [ ]:
aarc_sample_raw\
    [["PersonId", "DepartmentId", "Year", "Taxonomy"]]\
    .drop_duplicates()\
    .shape

In [ ]:
aarc_sample_raw.shape

- violations of the second and third requirements include
  - (2.) `PersonName` depending only on `PersonId`
  - (2.) `Rank` depending only on `(DepartmentId, PersonId, Year)`
  - (3.) `InstitutionName` depending on `InstitutionId` which depends on `DepartmentId` only (2.)
- see [appendix](#other-normal-forms) for details on other normal forms
  - in any real setting you typically go straight to 3NF 
  - but how?

### Entity-Relationship diagram
- first, map out the **entities** and **relationships** in your data in an **Entity-Relation diagram** ([ERD](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model))

```mermaid
erDiagram
    persons {
        int PersonId PK
        int DegreeInstitutionId FK
        string PersonName
        int DegreeYear
        string Gender
    }

    institutions {
        int InstitutionId PK
        string InstitutionName
    }

    departments {
        int DepartmentId PK
        string DepartmentName
    }

    taxonomies {
        int TaxonomyId PK
        string Taxonomy
    }

    fields {
        int FieldId PK
        string Field
    }

    areas {
        int AreaId PK
        string Area
    }

    umbrellas {
        int UmbrellaId PK
        string Umbrella
    }

    department_taxonomy {
        int DepartmentId PK, FK
        int TaxonomyId PK, FK
        int AreaId FK
        int FieldId FK
        int UmbrellaId FK
    }

    appointments {
        int PersonId PK, FK 
        int InstitutionId PK, FK 
        int DepartmentId PK, FK
        int Year PK 
        string Rank
        boolean PrimaryAppointment
        boolean Imputed
    }

    persons }o--o| institutions : "degree_from"

    appointments }|--|| persons : "has_appointment"
    appointments }o..o| institutions : "at_institution"
    appointments }|--|| departments : "in_department"

    department_taxonomy }o--|{ departments : "maps_department"
    department_taxonomy }o--|{ taxonomies : "maps_taxonomy"
    department_taxonomy }o--o{ fields : "maps_field"
    department_taxonomy }o--o{ areas : "maps_area"
    department_taxonomy }o--o{ umbrellas : "maps_umbrella"
```

Next, we split the giant table into separate tables for each entity and relationship.

### Transforming AARC to 3NF
The `normalize_data` function implements the transformation from the unnormalized table to a collection of tables.
It returns a custom `AARCCollection` object that stores the tables as members.

In [ ]:
aarc_nf = normalize_data(aarc_sample_raw)

In [ ]:
aarc_nf.appointments

To create entity tables, we simply aggregate the respective attributes by the respective primary key:

```python
    df_persons = df_data.groupby("PersonId")\
        .aggregate({
            "Gender": "first",
            "DegreeYear": "first",
            "PersonName": "first",
            "DegreeInstitutionId": "first",
    })
```

Relationship tables are created by aggregating over the compound primary keys:

```python
    df_appointments = df_data\
        .groupby(
            ["PersonId", "Year", "DepartmentId", "InstitutionId"],
            dropna=False)\
        .aggregate({
            "Rank": "first",
            "PrimaryAppointment": "first"
        })
```

Check the function's code to see details on how to create artificial IDs for entities that do not have a primary key yet (e.g., `Umbrellas`).

By avoiding most of the redundancy, the size of the entire dataset reduced from 3MB to 1MB.

### Simple statistics
From here, it's very easy to produce simple statistics on the entity and relationship tables without worrying about duplicates!
For instance, plotting the distribution of degree years per person:

In [ ]:
aarc_nf.persons["DegreeYear"].hist()
plt.xlabel("Degree Year")
plt.ylabel("Frequency")

In the previous dataset, we first had to remove duplicates, because faculty appear in multiple rows. This is error-prone.

### Joining
More complicated analysis require re-joining the separate tables back together.
For instance, to show the number of departments per umbrella, we need the umbrella names from `aarc_nf.umbrellas`.

In [ ]:
aarc_nf.department_taxonomies\
    .join(
        aarc_nf.umbrellas,
        on="UmbrellaId", # the foreign key in `department_taxonomies`
        how="left", # left join keeps all rows from `department_taxonomies`
        )\
    .groupby([
        "UmbrellaId", # in case of duplicate umbrella names
        "Umbrella"])\
    ["DepartmentId"]\
    .nunique()\
    .sort_index()

Let's compute the annual department sizes per gender as another example.

In [ ]:
aarc_nf.appointments\
    .join(aarc_nf.persons["Gender"],
          on="PersonId")\
    .groupby(["DepartmentId", "Gender", "Year"])\
    .size()\
    .sort_index()

Joins are a complex topic on their own.
For the start, you have to keep in mind missing data.

In [ ]:
aarc_nf.department_taxonomies.isna().sum()

In seven rows, the `FieldId` is not set. This has consequences when joining the respective `fields`-table, because you need to decide how these rows should be handled.
The default, a **left-join**, is to consider the first table (on which you call the `.join`-method) as the source table and keep all its rows.
Keys that could not be matched are filled with empty attributes.
If your data follows **foreign integrity** this only occurs for empty foreign-keys.

In [ ]:
l_join = aarc_nf.department_taxonomies.join(
    aarc_nf.fields,
    on="FieldId",
    how="left" # the default
)
l_join[l_join["FieldId"].isna()]

A **right-join** considers the joined table as source table instead and keeps all its rows. 
This ignores (!) rows in the other table for which no `FieldId` could be found.

In [ ]:
r_join = aarc_nf.department_taxonomies.join(
    aarc_nf.fields,
    on="FieldId",
    how="right"
)
r_join[r_join["FieldId"].isna()]

You can also do **inner-joins** or **outer-joins** to include no missing values or all missing values, respectively.

In [ ]:
len(aarc_nf.department_taxonomies.join(
    aarc_nf.fields,
    on="FieldId",
    how="outer"
))

Finally, `.join` always matches the values in the `on`-column to the index (!) of the reference table.
Panda's `.merge`-function is more powerful and allows to match arbitrarily on indices or columns of either table.

## Advanced topics

### Databases

Databases are often used when datasets are too big to be stored in local memory or when many people need to work on the same dataset.
All concepts naturally translate to databases (in fact it's what normalization was developed for initially).
The most common databases for relational data are based on the SQL-language.
Everything presented here naturally translates to these databases, where each `DataFrame` is represented as its own table.

In [ ]:
# Connect to a temporary SQLite database:
with sqlite3.connect(FOLDER_DB) as conn:
    # Creates tables based on the persons and appointments DataFrames:
    aarc_nf.persons.to_sql(
        "persons",
        conn,
        if_exists="replace",
        index=True,
    )
    aarc_nf.appointments.to_sql(
        "appointments",
        conn,
        if_exists="replace",
        index=True,
    )

    # Get number of faculty members broken down by gender, department, and year:
    res = conn.execute("""
        SELECT
            p.gender,
            a.departmentid,
            a.year,
            COUNT(DISTINCT p.personid) AS count
        FROM appointments AS a
        JOIN persons AS p
        ON a.personid = p.personid
        GROUP BY
            a.departmentid,
            a.year"""
    ).fetchall() # Get all results
res

### Other normal forms

#### 1. Normal Form (1NF)
- each cell consists of **atomic** elements only
  - no collections of values are allowed
- the `AARC-v3` dataset already follows this normal form
- let's consider an artificial counter example

In [ ]:
unf = pd.DataFrame(
    data={
        "DepartmentId": [1, 2, 3],
        "Year": [2020, 2021, 2022],
        "DepartmentName": ["A", "B", "C"],
        "PersonId": [[1,2], [3,4,5], [10]],
        "PrestigeRank": [.1, .1, .2],
        "DepartmentHead": ["Bob", "Alice", "Charlie"],
        "DepartmentHeadGender": ["M", "W", "M"],
    }
)
unf

This can be transformed to 1NF by 
- **exploding** the `PersonId` column (the name of the functions matches its implications)

In [ ]:
aarc_toy = unf.explode("PersonId")
aarc_toy

- this is the current state of the `AARC-v3` dataset!
  - note the redundancy in `DepartmentName` and the lack of identifiability
- better: separation into multiple tables
  - indexed by their **primary key**

In [ ]:
aarc_toy_hiring = aarc_toy\
    [["DepartmentId", "PersonId", "Year", "PrestigeRank"]]\
    .drop_duplicates()\
    .set_index(["DepartmentId", "PersonId", "Year"])\
    .sort_index()
aarc_toy_hiring

In [ ]:
aarc_toy_dep = aarc_toy\
    [["DepartmentId", "DepartmentName", "DepartmentHead", "DepartmentHeadGender"]]\
    .drop_duplicates()\
    .set_index("DepartmentId")
aarc_toy_dep

#### 2. Normal Form (2NF)
- 2NF requires that all columns depend on the full **primary key**
- `aarc_toy`, `aarc_toy_hiring` and the full sample `aarc_sample_raw` all violate this requirement
  - for instance, `PrestigeRank` in `aarc_toy_hiring` depends only on `DepartmentId` and `Year`, but not on `PersonId`
  - there are many more examples in the full sample
    - `DepartmentName` depends only on `DepartmentId` not on the other key parts (`PersonId`, `Year`, `InstitutionId`, ...)

In [ ]:
aarc_toy[["DepartmentId", "PersonId", "Year"]].drop_duplicates()

In [ ]:
aarc_toy[["DepartmentId", "Year", "PrestigeRank"]].drop_duplicates().set_index(
    ["DepartmentId", "Year"]
).sort_index()

#### 3. Normal Form (3NF)
- 3NF requires that there are no transitive dependencies
- in our toy example, `aarc_toy_dep` still violates this because `DepartmentHeadGender` depends on `DepartmentHead` which depends on `DepartmentId`

In [ ]:
aarc_toy_dep.reset_index()[["DepartmentHead", "DepartmentHeadGender"]].drop_duplicates()

In [ ]:
aarc_toy_dep[["DepartmentHead", "DepartmentName"]].drop_duplicates()

- we created four separate tables, each representing a unique entity or relationship
- there is no redundant data and each entity/relationship is identifiable